# uDALES Urban Surface Creation with Python

uDALES supports several methods to create urban surfaces. It is possible to run uDALES without specifying an urban surface (flat terrain is assumed), but in most cases an urban surface will be specified.

This tutorial covers:

### Geometry Generation Functions
- **create_flat_surface**: Create flat surface with triangular facets
- **create_canyons**: Create one-dimensional street canyons
- **create_cubes**: Create cubes (single or arrays)
- **create_realistic**: Create realistic urban surface from buildings + ground

### The UDGeom Class
- **load**: Load STL file
- **save**: Save STL file  
- **show**: Display geometry in 3D
- Properties: n_faces, n_vertices, bounds, volume, total_area, etc.

## 1. Import Libraries and Setup

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Add the uDALES python path
sys.path.insert(0, '../tools/python')

from udgeom import UDGeom
from geometry_generation import (
    create_flat_surface,
    create_canyons,
    create_cubes,
    create_realistic
)

## 2. The UDGeom Class

The UDGeom class represents uDALES urban surfaces and provides methods to load, save, and visualize STL files.

In [ ]:
# Example: create a udgeom instance
geom = UDGeom()

# Show available methods
help(geom.load)
help(geom.save)
help(geom.show)

In [ ]:
help(UDGeom)

In [ ]:
# Create a simple geometry example
geom = create_flat_surface(96, 96, edgelength=12)

print(f"Faces: {geom.n_faces}")
print(f"Vertices: {geom.n_vertices}")
print(f"Bounds: {geom.bounds}")
print(f"Total area: {geom.total_area:.2f} m²")

# Visualize
geom.show()
plt.xlabel('x [m]')
plt.ylabel('y [m]')

## 3. create_flat_surface: Creating Flat Surfaces

In [ ]:
help(create_flat_surface)

In [ ]:
# Create flat surface
xsize = 96
ysize = 96
edgelength = xsize / 8  # 12m facets

geom = create_flat_surface(xsize, ysize, edgelength)

print(f"Domain: {xsize}m × {ysize}m")
print(f"Number of faces: {geom.n_faces}")
print(f"Total area: {geom.total_area:.2f} m²")

# Save to file
geom.save('flat_surface.stl')

## 4. create_canyons: Creating Street Canyons

In [ ]:
help(create_canyons)

In [ ]:
# Create street canyons
xsize = 96
ysize = 96
B = 12  # building width
H = 16  # building height
W = 12  # street width

geom = create_canyons(
    xsize=xsize, ysize=ysize,
    Hx=B, Hy=ysize, Hz=H,
    Cx=W, Cy=0,
    orientation='y',
    edgelength=6,
    add_ground=True
)

print(f"Building: {B}m × {ysize}m × {H}m")
print(f"Street width: {W}m")
print(f"H/W ratio: {H/W:.2f}")
print(f"Number of faces: {geom.n_faces}")
print(f"Volume: {geom.volume:.2f} m³")

geom.save('canyon.stl')

## 5. create_cubes: Creating Cube Geometries

In [ ]:
help(create_cubes)

In [ ]:
# Single cube
geom_single = create_cubes(96, 96, 16, 16, 16, geom_option='S', edgelength=8)
print(f"Single cube: {geom_single.n_faces} faces")
geom_single.save('single_cube.stl')

# Aligned array
geom_aligned = create_cubes(96, 96, 16, 16, 16, Cx=8, Cy=8, 
                            geom_option='AC', edgelength=8)
print(f"Aligned array: {geom_aligned.n_faces} faces")
geom_aligned.save('aligned_cubes.stl')

# Staggered array
geom_staggered = create_cubes(96, 96, 16, 16, 16, Cx=8, Cy=8,
                              geom_option='SC', edgelength=8)
print(f"Staggered array: {geom_staggered.n_faces} faces")
geom_staggered.save('staggered_cubes.stl')

## 6. create_realistic: Realistic Urban Layouts

In [ ]:
help(create_realistic)

In [ ]:
# Create realistic layout from building list
buildings = [
    {'x': 10, 'y': 10, 'width': 20, 'length': 20, 'height': 15},
    {'x': 40, 'y': 10, 'width': 15, 'length': 25, 'height': 20},
    {'x': 10, 'y': 45, 'width': 25, 'length': 15, 'height': 18},
    {'x': 50, 'y': 50, 'width': 20, 'length': 20, 'height': 12},
]

geom = create_realistic(
    buildings=buildings,
    domain_size=(96, 96),
    edgelength=8,
    add_ground=True
)

print(f"Number of buildings: {len(buildings)}")
print(f"Number of faces: {geom.n_faces}")
print(f"Volume: {geom.volume:.2f} m³")

geom.save('realistic.stl')

## 7. Workflow and Best Practices

### Typical Workflow

1. **Choose geometry type** based on your research question
   - Flat surface: Simple benchmark cases
   - Canyons: 2D flow studies, idealized urban canyons
   - Cubes: Array studies, parametric investigations
   - Realistic: Real urban layouts, specific locations

2. **Create geometry** using appropriate function

3. **Inspect properties**
   ```python
   print(f"Faces: {geom.n_faces}")
   print(f"Volume: {geom.volume:.2f} m³")
   print(f"Area: {geom.total_area:.2f} m²")
   ```

4. **Visualize** to verify correctness
   ```python
   geom.show()
   plt.show()
   ```

5. **Save to STL** for use in simulations
   ```python
   geom.save('geometry.stl')
   ```

### Important Considerations

- **Facet size**: Ground facet size affects mesh resolution. Smaller = more detail but more facets
- **Domain size**: Ensure periodic boundaries work correctly (buildings should fit evenly)
- **Height**: First grid cell above buildings should have sufficient resolution
- **Coordinates**: Origin (0,0) is typically domain corner; buildings positioned relative to this

## 8. Summary

### Geometry Generation Functions

| Function | Use Case | Key Parameters |
|----------|----------|----------------|
| `create_flat_surface` | Flat ground | xsize, ysize, edgelength |
| `create_canyons` | Street canyons | Building dims, spacing, orientation |
| `create_cubes` | Cube arrays | Cube size, spacing, option (S/AC/SC) |
| `create_realistic` | Real layouts | Buildings list, domain_size |

### UDGeom Class Capabilities
- ✅ Load/save STL files
- ✅ 3D visualization
- ✅ Geometry properties (area, volume, bounds)
- ✅ Facet-level data (centers, normals, areas)

### Next Steps
- **udbase_tutorial.ipynb** - Introduction to post-processing
- **fields_tutorial.ipynb** - Working with field data
- **facets_tutorial.ipynb** - Surface energy balance analysis
- `tools/python/geometry_example.py` - Complete geometry examples
- uDALES documentation: https://u-dales.readthedocs.io/

### Quick Reference
```python
# Create geometry
geom = create_cubes(96, 96, 16, 16, 16, Cx=8, Cy=8, geom_option='AC')

# Inspect
print(f"Faces: {geom.n_faces}, Volume: {geom.volume:.1f} m³")

# Visualize
geom.show()

# Save
geom.save('mygeometry.stl')
```